### Import necessary Libraries

In [19]:
import torch
from torch import nn
import torchvision
from torchinfo import summary
from torchvision import datasets
from pathlib import Path
import os

In [20]:
NUM_WORKERS = 2 if os.cpu_count() <= 4 else 4

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
device

'cuda'

### Load our EffnetB2 Model

In [4]:
def create_effnetb2(num_classes: int=101, seed: int=42):
    effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    effnetb2 = torchvision.models.efficientnet_b2(effnetb2_weights)
    effnetb2_transforms = effnetb2_weights.transforms()

    for param in effnetb2.parameters():
        param.requires_grad = False

    torch.manual_seed(42)
    effnetb2.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes)
    )

    return effnetb2, effnetb2_transforms

In [5]:
effnetb2_food101, effnetb2_transforms = create_effnetb2()

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [6]:
summary(effnetb2_food101,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 101]             --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

### Downloading Data & Preprocessing

torchvision.transforms.TrivialAugmentWide() is the same data augmentation used by the pytorch team in their Computer Vision Recipe

In [7]:
food101_train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.TrivialAugmentWide(),
    effnetb2_transforms
])

In [11]:
Path("data/food-101").exists() and Path("data/food-101/license_agreement.txt").is_file()

True

In [15]:
data_dir = Path("data")
if Path(data_dir / "food-101").exists() and Path("data/food-101/license_agreement.txt").is_file():
    train_data = datasets.Food101(
        root=data_dir, split="train", transform=food101_train_transforms, download=False)
    test_data = datasets.Food101(
        root=data_dir, split="test", transform=effnetb2_transforms, download=False)
else:
    train_data = datasets.Food101(root=data_dir, split="train", transform=food101_train_transforms, download=True)

    test_data = datasets.Food101(root=data_dir, split="test", transform=effnetb2_transforms, download=True)

In [16]:
food101_class_names = train_data.classes
food101_class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

### Training on 20% of the full dataset

In [18]:
def split_dataset(dataset: torchvision.datasets, split_size: float=0.2, seed:int=42):
    length_1 = int(len(dataset) * split_size)
    length_2 = len(dataset) - length_1

    print(f"[INFO] Splitting dataset of length {len(dataset)} into splits of size {length_1} ({int(split_size*100)}%), {length_2} ({int((1-split_size)*100)}%)")

    random_split_1, random_split_2 = torch.utils.data.random_split(dataset, lengths=[length_1, length_2], generator=torch.manual_seed(seed))

    return random_split_1, random_split_2

well, we will proceed to train on the full dataset

### Data Preprocessing

In [27]:
BATCH_SIZE = 32
train_dataloader_food101 = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader_food101 = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

In [31]:
test_dataloader_food101.dataset, train_dataloader_food101.dataset

(Dataset Food101
     Number of datapoints: 25250
     Root location: data
     split=test
     StandardTransform
 Transform: ImageClassification(
                crop_size=[288]
                resize_size=[288]
                mean=[0.485, 0.456, 0.406]
                std=[0.229, 0.224, 0.225]
                interpolation=InterpolationMode.BICUBIC
            ),
 Dataset Food101
     Number of datapoints: 75750
     Root location: data
     split=train
     StandardTransform
 Transform: Compose(
                TrivialAugmentWide(num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
                ImageClassification(
                crop_size=[288]
                resize_size=[288]
                mean=[0.485, 0.456, 0.406]
                std=[0.229, 0.224, 0.225]
                interpolation=InterpolationMode.BICUBIC
            )
            ))

### Training

In [ ]:
optimizer = torch.optim.Adam(params=effnetb2_food101.parameters(), lr=1e-3)
loss = torch.nn.CrossEntropyLoss(label_smoothing=0.1)